# 02 - Data Cleaning

This notebook cleans the raw metadata based on EDA findings.

## Cleaning Steps
- Handle missing values
- Remove duplicates
- Standardize date formats
- Validate and normalize text fields
- Filter out unusable records

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import re

import pandas as pd
import numpy as np

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.config import get_settings, ensure_directories

In [ ]:
# Load settings and data
settings = get_settings()
ensure_directories(settings)

data_path = settings.raw_data_dir / settings.raw_metadata_file
print(f"Loading raw data from: {data_path}")

df = pd.read_csv(data_path)
print(f"Loaded {len(df):,} records")
print(f"Columns: {list(df.columns)}")

In [ ]:
# Initial state
initial_count = len(df)
print(f"\nInitial record count: {initial_count:,}")

## 1. Remove Duplicates

In [ ]:
# Remove exact duplicates
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Removed {before - after:,} exact duplicates")

# Remove duplicate identifiers (keep first)
before = len(df)
df = df.drop_duplicates(subset=['identifier'], keep='first')
after = len(df)
print(f"Removed {before - after:,} duplicate identifiers")

In [ ]:
# Remove duplicate titles (more aggressive - optional)
# Uncomment if you want to remove records with duplicate titles

# before = len(df)
# df = df.drop_duplicates(subset=['title'], keep='first')
# after = len(df)
# print(f"Removed {before - after:,} duplicate titles")

## 2. Handle Missing Values

In [ ]:
# Check current missing values
print("Current missing values:")
for col in df.columns:
    missing = df[col].isna().sum()
    if missing > 0:
        print(f"  {col}: {missing:,} ({missing/len(df)*100:.1f}%)")

In [ ]:
# Remove records without title (essential field)
before = len(df)
df = df[df['title'].notna() & (df['title'].str.strip() != '')]
after = len(df)
print(f"Removed {before - after:,} records without title")

In [ ]:
# Remove records without abstract (needed for topic modeling)
before = len(df)
df = df[df['abstract'].notna() & (df['abstract'].str.strip() != '')]
after = len(df)
print(f"Removed {before - after:,} records without abstract")

In [ ]:
# Fill missing values for non-essential fields
df['authors'] = df['authors'].fillna('')
df['subjects'] = df['subjects'].fillna('')
df['publisher'] = df['publisher'].fillna('')
df['types'] = df['types'].fillna('')
df['language'] = df['language'].fillna('')
df['source'] = df['source'].fillna('')

print("Filled missing values for non-essential fields")

## 3. Standardize Date Format

In [ ]:
# Parse and standardize dates
def parse_date(date_str):
    """Parse various date formats and extract year."""
    if pd.isna(date_str) or date_str == '':
        return None, None
    
    date_str = str(date_str).strip()
    
    # Try to parse as datetime
    try:
        parsed = pd.to_datetime(date_str)
        return parsed, parsed.year
    except:
        pass
    
    # Extract year using regex
    year_match = re.search(r'(19|20)\d{2}', date_str)
    if year_match:
        year = int(year_match.group())
        return pd.Timestamp(year=year, month=1, day=1), year
    
    return None, None

# Apply parsing
parsed_dates = df['date'].apply(parse_date)
df['parsed_date'] = [d[0] for d in parsed_dates]
df['year'] = [d[1] for d in parsed_dates]

# Statistics
valid_dates = df['year'].notna().sum()
print(f"Successfully parsed dates: {valid_dates:,} ({valid_dates/len(df)*100:.1f}%)")

In [ ]:
# Year distribution
print("\nYear distribution:")
year_counts = df['year'].value_counts().sort_index()
print(year_counts.tail(10))

## 4. Text Normalization

In [ ]:
def normalize_text(text):
    """Normalize text field."""
    if pd.isna(text) or text == '':
        return ''
    
    text = str(text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    # Remove control characters
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    
    return text

# Apply normalization
df['title'] = df['title'].apply(normalize_text)
df['abstract'] = df['abstract'].apply(normalize_text)
df['authors'] = df['authors'].apply(normalize_text)
df['subjects'] = df['subjects'].apply(normalize_text)

print("Applied text normalization to title, abstract, authors, subjects")

## 5. Filter Records by Abstract Length

In [ ]:
# Calculate word counts
df['abstract_word_count'] = df['abstract'].str.split().str.len()

print("Abstract word count statistics:")
print(df['abstract_word_count'].describe())

In [ ]:
# Remove very short abstracts
MIN_ABSTRACT_WORDS = 20

before = len(df)
df = df[df['abstract_word_count'] >= MIN_ABSTRACT_WORDS]
after = len(df)

print(f"Removed {before - after:,} records with abstract < {MIN_ABSTRACT_WORDS} words")

## 6. Final Cleanup

In [ ]:
# Reset index
df = df.reset_index(drop=True)

# Select final columns
final_columns = [
    'identifier',
    'title',
    'abstract',
    'authors',
    'date',
    'year',
    'subjects',
    'publisher',
    'types',
    'language',
]

# Keep only existing columns
final_columns = [c for c in final_columns if c in df.columns]
df_clean = df[final_columns].copy()

print(f"Final columns: {list(df_clean.columns)}")

In [ ]:
# Summary
print("\n" + "=" * 60)
print("CLEANING SUMMARY")
print("=" * 60)
print(f"Initial records:  {initial_count:,}")
print(f"Final records:    {len(df_clean):,}")
print(f"Removed:          {initial_count - len(df_clean):,} ({(initial_count - len(df_clean))/initial_count*100:.1f}%)")
print("=" * 60)

## 7. Save Cleaned Data

In [ ]:
# Save to processed directory
output_path = settings.processed_data_dir / settings.clean_metadata_file

df_clean.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✅ Saved cleaned data to: {output_path}")
print(f"📊 Total records: {len(df_clean):,}")

In [ ]:
# Preview cleaned data
df_clean.head()

In [ ]:
# Quick stats
print("\nCleaned data statistics:")
print("-" * 40)
print(f"Records: {len(df_clean):,}")
print(f"Date range: {df_clean['year'].min():.0f} - {df_clean['year'].max():.0f}")
print(f"Avg abstract words: {df_clean['abstract'].str.split().str.len().mean():.0f}")

In [ ]:
print(f"\n👉 Next: Run 02b_eda_clean_data.ipynb to analyze the cleaned data")
print(f"   Then: Run 03_preprocessing.ipynb for text preprocessing")